In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/augmented-eyes-4/dataframe_bueno.pkl
/kaggle/input/augmented-eyes-4/y_train.npy
/kaggle/input/augmented-eyes-4/X2_train.npy
/kaggle/input/augmented-eyes-4/name_file_test.npy
/kaggle/input/augmented-eyes-4/y_test.npy
/kaggle/input/augmented-eyes-4/__results__.html
/kaggle/input/augmented-eyes-4/name_file_train.npy
/kaggle/input/augmented-eyes-4/dataframe.pkl
/kaggle/input/augmented-eyes-4/X2_test.npy
/kaggle/input/augmented-eyes-4/X1_train.npy
/kaggle/input/augmented-eyes-4/__notebook__.ipynb
/kaggle/input/augmented-eyes-4/X1_test.npy
/kaggle/input/augmented-eyes-4/__output__.json
/kaggle/input/augmented-eyes-4/X0_test.npy
/kaggle/input/augmented-eyes-4/X0_train.npy
/kaggle/input/augmented-eyes-4/custom.css
/kaggle/input/augmented-eyes-4/__results___files/__results___13_0.png


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Input, Conv2D, multiply, LocallyConnected2D, Lambda
from keras.models import Model
from keras.layers import BatchNormalization, concatenate
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

2024-02-24 19:58:05.161325: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-24 19:58:05.161429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-24 19:58:05.293992: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
%pip install -q -U keras-tuner

Note: you may need to restart the kernel to use updated packages.


In [4]:
width = 224
height = 224
batch_size = 32

In [5]:
def load_data(file_x0,file_x1,file_x2, file_y):
    """
        Reading numpy data files
    """

    X0 = np.load('/kaggle/input/augmented-eyes-4/'+file_x0)
    X1 = np.load('/kaggle/input/augmented-eyes-4/'+file_x1)
    X2 = np.load('/kaggle/input/augmented-eyes-4/'+file_x2)
    y = np.load('/kaggle/input/augmented-eyes-4/'+file_y)
    
    ohe = OneHotEncoder()
    X2 = ohe.fit_transform(X2.reshape(-1,1)).toarray()
    y = ohe.fit_transform(y.reshape(-1,1)).toarray()
     
    return (X0 , X1, X2, y)

In [6]:
#  Load data, and change type for speed

(X0_train, X1_train, X2_train, y_train)= load_data('X0_train.npy', 'X1_train.npy',
                                                   'X2_train.npy', 'y_train.npy')
(X0_test, X1_test, X2_test, y_test)= load_data('X0_test.npy', 'X1_test.npy',
                                                   'X2_test.npy', 'y_test.npy')

X0_train = X0_train.astype('uint8')
X0_test = X0_test.astype('uint8')

X1_train = X1_train.astype('uint8')
X1_test = X1_test.astype('uint8')

X2_train = X2_train.astype('uint8')
X2_test = X2_test.astype('uint8')

y_test = y_test.astype('uint8')
y_train = y_train.astype('uint8')

In [7]:
# Reshape ages to be a 2D array
X1_train = X1_train.reshape(-1, 1)
X1_test = X1_test.reshape(-1, 1)

# Combine the arrays
combined_train = np.hstack((X1_train, X2_train))
combined_test = np.hstack((X1_test, X2_test))

In [8]:
#Hyperparamter tuning for number of hidden layers and neurons and the learning rate

import keras_tuner as kt
def build_model(hp):
    
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=3, default=2)
    n_hidden2 = hp.Int("n_hidden2", min_value=0, max_value=4, default=2)
    n_neurons_1 = hp.Int("n_neurons", min_value=1, max_value=32)
    n_neurons_2 = hp.Int("n_neurons2", min_value=1, max_value=64)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2,
                             sampling="log")
    
    dropout_rate = hp.Float("dropout_rate", min_value=0.25, max_value=0.8)
    dropout_rate2 = hp.Float("dropout_rate2", min_value=0.1, max_value=0.75)
    
    optimizer = hp.Choice("optimizer", values=["sgd", "adam"])
    if optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, 
                                            momentum=0.9, nesterov=True)
    else:
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,
                                             beta_1=0.9, beta_2=0.999, epsilon=1e-7)
    
    # MLP Model
    
    mlp_input = Input(shape=(3,))
    mlp = mlp_input  # Initialize mlp here
    for _ in range(n_hidden):
        mlp = Dense(units=n_neurons_1, activation='relu')(mlp)
    mlp = Dense(units=1, activation='relu')(mlp)
    
    # CNN Model
    input_l = Input(X0_train.shape[1:])
    base_model = VGG16(input_shape =  X0_train.shape[1:], include_top = False, weights = 'imagenet')
    base_model.trainable = True
    pt_features = base_model(input_l)
    bn_features = BatchNormalization()(pt_features)
    pt_depth = base_model.get_output_shape_at(0)[-1]

    attn_layer = Conv2D(64, kernel_size = (1,1), padding = 'same', 
                        activation = 'relu')(Dropout(dropout_rate)(bn_features))
    attn_layer = Conv2D(16, kernel_size = (1,1), padding = 'same', 
                        activation = 'relu')(attn_layer)
    attn_layer = Conv2D(8, kernel_size = (1,1), padding = 'same', 
                        activation = 'relu')(attn_layer)
    attn_layer = Conv2D(1, kernel_size = (1,1), padding = 'valid', 
                        activation = 'sigmoid')(attn_layer)

    up_c2_w = np.ones((1, 1, 1, pt_depth))
    up_c2 = Conv2D(pt_depth, kernel_size = (1,1), padding = 'same', 
                   activation = 'linear', use_bias = False, weights = [up_c2_w])
    up_c2.trainable = False
    attn_layer = up_c2(attn_layer)


    mask_features = multiply([attn_layer, bn_features])
    gap_features = GlobalAveragePooling2D()(mask_features)
    gap_mask = GlobalAveragePooling2D()(attn_layer)

    # to account for missing values from the attention model
    gap = Lambda(lambda x: x[0]/x[1], name = 'RescaleGAP')([gap_features, gap_mask])
    gap_dr = Dropout(dropout_rate2)(gap)
    dr_steps = Dropout(dropout_rate2)(Dense(128, activation = 'relu')(gap_dr))
    #cnn_flatten = Flatten()(dr_steps)
    
    combined_layer = concatenate([mlp, dr_steps])
    dense = combined_layer  # Initialize dense here
    for _ in range(n_hidden2):
        dense = Dense(units=n_neurons_2, activation='relu')(dense)
    out_layer = Dense(y_train.shape[-1], activation='softmax')(dense)

    model = Model(inputs=[mlp_input, input_l], outputs=out_layer)
    
    metrics = [tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                   tf.keras.metrics.Precision(name='precision'),
                   tf.keras.metrics.Recall(name='recall')]

    model.compile(optimizer = optimizer, loss = 'categorical_crossentropy',
                           metrics = metrics)
    return model
    

In [ ]:
# Add some constraints and search
weight_path="{}_weights.best.hdf5".format('retina')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)
callbacks_list = [checkpoint, early, reduceLROnPlat]

hyperband = kt.Hyperband(
    build_model, objective="val_accuracy",
    max_epochs=10, factor=3, hyperband_iterations=2,
    overwrite=True, directory="/kaggle/working/", project_name="hyperband")

history = hyperband.search(x=[combined_train, X0_train], y=y_train, epochs=12, verbose=1,
                                               callbacks=callbacks_list,
                                               shuffle=True, validation_data=([combined_test, X0_test], y_test))

Trial 30 Complete [00h 26m 58s]
val_accuracy: 0.6581140160560608

Best val_accuracy So Far: 0.8002192974090576
Total elapsed time: 05h 31m 35s

Search: Running Trial #31

Value             |Best Value So Far |Hyperparameter
3                 |3                 |n_hidden
1                 |0                 |n_hidden2
31                |7                 |n_neurons
36                |57                |n_neurons2
0.0018682         |0.00042746        |learning_rate
0.51271           |0.46634           |dropout_rate
0.45365           |0.3767            |dropout_rate2
adam              |sgd               |optimizer
2                 |10                |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |0                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


2024-02-25 01:31:01.912970: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


 91/570 [===>..........................] - ETA: 2:07 - loss: 2.0829 - accuracy: 0.1569 - precision: 0.4000 - recall: 0.0014

In [ ]:
# Save the best model
top3_models = random_search_tuner.get_best_models(num_models=3)
best_model = top3_models[0]

In [ ]:
# Get the best hyperparameters
top3_params = random_search_tuner.get_best_hyperparameters(num_trials=3)
top3_params[0].values